In [1]:
cd C:/Users/mstark/Desktop/uw-msds-trac-capstone/data/training_data/

C:\Users\mstark\Desktop\uw-msds-trac-capstone\data\training_data


In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import LinearSVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
folders = ['winter_data/aggregates', 'summer_data/aggregates', 'combined_data'] 
aggregations = ['15', '30', 'hr']

In [7]:
Model = []
MAE_test = []
MAE_val = []
for f in folders:
    for a in aggregations:
        c = 3
        
        Model.append([f, a])
        print('Running:', c, 'cluster SVR on ', f, 'data w/', a, 'min aggregation')
        filepath = f + '/' + a + 'min/'
        print(f[0:3])
        train_set = pd.read_csv(filepath + 'train.tsv.gz', sep='\t')
        test_set = pd.read_csv(filepath + 'test.tsv.gz', sep='\t')
        val_set = pd.read_csv(filepath + 'xval.tsv.gz', sep='\t')
        
        cols = list(train_set.columns)
        cols.remove('is_rapid')
        cols.remove('opd_date')
        cols.remove('ons')
        categorical_cols = ['dir', 'trip_start_hr_' + a , 'type', 'start', 'end', 'region']

        X_train = train_set[cols]
        Y_train = train_set['ons']
        X_test = test_set[cols]
        Y_test = test_set['ons']
        X_val = val_set[cols]
        Y_val = val_set['ons']
        all_cols = cols + ['ons']
        
        full_set = train_set.append(test_set.append(val_set))[all_cols]
        print('Passenger count (orca, apc):')
        print(full_set[['orca_total', 'ons']].sum())
        
        le = LabelEncoder()

        for cc in categorical_cols:
            le.fit(train_set[cc])
#             full_set[cc] = le.transform(full_set[cc])
            X_train[cc] = le.transform(X_train[cc])
            X_test[cc] = le.transform(X_test[cc])
            X_val[cc] = le.transform(X_val[cc])
        if c == 1:
            print('skip clustering')
        else:
            print('clustering')
            kmeans = KMeans(n_clusters=c, random_state=0).fit(X_train)
            X_train['group'] = pd.Series(kmeans.labels_)
            X_test['group'] = pd.Series(kmeans.predict(X_test))
            X_val['group'] = pd.Series(kmeans.predict(X_val))
        
        for cluster in range(0, c):
            
            if c == 1:
                X_0 = X_train[cols]
                Y_0 = Y_train
            else:
                X_0 = X_train[X_train['group'] == cluster][cols]
                Y_0 = Y_train[X_train['group'] == cluster]
            
            print('modeling')
            regrs0 = LinearSVR(max_iter = 5000, tol = 1e-6)
            regrs0.fit(X_0, Y_0)
            
            X_test['predictions'] = regrs0.predict(X_test[cols])
            X_val['predictions'] = regrs0.predict(X_val[cols])
            
            if c == 1:
                X_test['final'] = X_test['predictions']
                X_val['final'] = X_val['predictions']
            elif cluster == 0:
                X_test['final'] = X_test['predictions'].where(X_test['group'] == cluster, None)
                X_val['final'] = X_val['predictions'].where(X_val['group'] == cluster, None)
            else:
                X_test['final'] = X_test['predictions'].where(X_test['group'] == cluster, X_test['final'])
                X_val['final'] = X_val['predictions'].where(X_val['group'] == cluster, X_val['final'])
                
        error_test = np.mean(np.abs(Y_test - X_test['final']))
        MAE_test.append(error_test)
        print('Test MAE for all rtes = ', error_test)
        error_val = np.mean(np.abs(Y_val - X_val['final']))
        MAE_val.append(error_val)
        print('Validation MAE for all rtes = ', error_val)
        
        output_path = 'C:/Users/mstark/Desktop/uw-msds-trac-capstone/model/'
        MyList = list(X_val['final'].astype(str))
    
        MyFile=open(output_path + str(c) + 'ClusterSVR' + '_' + a + 'min_' + f[0:3] + '.txt','w')
        for element in MyList:
            MyFile.write(element + '\n')
        MyFile.close() 

Running: 3 cluster SVR on  winter_data/aggregates data w/ 15 min aggregation
win
Passenger count (orca, apc):
orca_total     6168912.0
ons           11175592.0
dtype: float64


C:\Users\mstark\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mstark\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mstark\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

clustering


C:\Users\mstark\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mstark\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mstark\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

modeling


C:\Users\mstark\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mstark\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


modeling
modeling
Test MAE for all rtes =  10.505297394129808
Validation MAE for all rtes =  10.419510898336403
Running: 3 cluster SVR on  winter_data/aggregates data w/ 30 min aggregation
win
Passenger count (orca, apc):
orca_total     6168912.0
ons           11175592.0
dtype: float64
clustering
modeling
modeling
modeling
Test MAE for all rtes =  12.000406453549003
Validation MAE for all rtes =  12.13776611396129
Running: 3 cluster SVR on  summer_data/aggregates data w/ 15 min aggregation
sum
Passenger count (orca, apc):
orca_total     6975490.0
ons           15049216.0
dtype: float64
clustering
modeling
modeling
modeling
Test MAE for all rtes =  11.3901248306519
Validation MAE for all rtes =  11.287546605534766
Running: 3 cluster SVR on  summer_data/aggregates data w/ 30 min aggregation
sum
Passenger count (orca, apc):
orca_total     6975490.0
ons           15049216.0
dtype: float64
clustering
modeling
modeling
modeling
Test MAE for all rtes =  17.024264629406765
Validation MAE for a

In [8]:
print(Model)
print(MAE_test)
print(MAE_val)

[['winter_data/aggregates', '15'], ['winter_data/aggregates', '30'], ['summer_data/aggregates', '15'], ['summer_data/aggregates', '30'], ['combined_data', '15'], ['combined_data', '30']]
[10.505297394129808, 12.000406453549003, 11.3901248306519, 17.024264629406765, 11.372595577171886, 13.067553683136559]
[10.419510898336403, 12.13776611396129, 11.287546605534766, 16.944444794854352, 11.327556887793508, 13.211547842536403]
